In [ ]:
# default_exp webservice

# Web service
> a web service to capture image and give prediction + card text

In [ ]:
#export
import sys, os
from pathlib import Path
import aiohttp
import pickle
import base64
from io import BytesIO

import uvicorn
from starlette.applications import Starlette
from starlette.responses import JSONResponse, RedirectResponse
from starlette.routing import Route, Mount
from starlette.templating import Jinja2Templates
from starlette.staticfiles import StaticFiles

from fastai.vision import *

In [ ]:
#export
ON_JUPY = os.environ['_'].split('/')[-1].lower() == 'jupyter'
static_dir = Path('pokedexr/static') if ON_JUPY else Path('static')

In [ ]:
#export
templates = Jinja2Templates(directory=static_dir / 'templates')
static_files = StaticFiles(directory=static_dir)
data_files = pathlib.Path(static_dir / 'data')

Load required data for inference

In [ ]:
#export
print('Initialising Learner...')
learn = load_learner(data_files,'export.pkl')
with open(data_files / 'cards.pkl','rb') as f:
    cards = pickle.load(f)

In [ ]:
#export 


[
{'number': '120/131', 'name': 'Volkner', 'details_page_ref': 'Volkner_(GX_Starter_Deck_120)', 'sname': 'Volkner', 'img_urls': ['//cdn.bulbagarden.net/upload/thumb/b/b6/VolknerUltraPrism135a.jpg/180px-VolknerUltraPrism135a.jpg', '//cdn.bulbagarden.net/upload/thumb/e/e1/VolknerUltraPrism135.jpg/200px-VolknerUltraPrism135.jpg', '//cdn.bulbagarden.net/upload/thumb/d/d2/VolknerUltraPrism156.jpg/200px-VolknerUltraPrism156.jpg'], 'card_text': [{'type': 'info', 'description': 'Search your deck for an Item card and a  Energy card, reveal them, and put them into your hand. Then, shuffle your deck.'}, {'type': 'info', 'description': 'You may play only 1 Supporter card during your turn (before your attack).'}], 'alt_card_num': {'alt_card_num': {'jp': ['034/038', '120/131', '025/029', '153/173', '095/095', '062/066', '029/035', '133/150', '071/066'], 'en': ['135/156', '156/156', '135a/156']}}}
]


In [ ]:
#export
async def home(request):
    return templates.TemplateResponse('index.html', {'request': request})

In [ ]:
#export
async def card(request):
    if request.path_params['name'] == 'notknown':
        return templates.TemplateResponse('cardnotknown.html', {'request': request})
    
    info = cards.get(request.path_params['name'],None)
    return templates.TemplateResponse('card.html', {'request': request, 'card':info})
    

In [ ]:
# export
async def feedback(request):
    ans,name = request.query_params['ans'],request.query_params['name']
    with open('feedback.csv','a') as f:       
        if ans == 'correct': 
            f.write(f"{name},1\n")
        elif ans == 'wrong':
            f.write(f"{name},0\n")
        else:
            pass
    return RedirectResponse(url=f"/", status_code=303)
        

In [ ]:
#export
import pdb
async def analyze(request):
    img_data = await request.form()
    img = open_image(img_data['file'].file)
    print('Got image, making prediction...')
    
    
    predicted_class, pred_class_i, class_probs = learn.predict(img)
    
    info = cards.get(str(predicted_class),None)
    if info:
        return RedirectResponse(url=f"/card/{info['sname']}", status_code=303)
    else:
        return RedirectResponse(url="/card/notknown")
    

In [ ]:
#export

routes = [
    Route('/', endpoint=home),
    Route('/card/{name}', card),
    Route('/analyze', analyze, methods=['POST']),
    Route('/feedback', feedback),
    Mount('/static', static_files, name='static'),
    
]

In [ ]:
#export
app = Starlette(debug=True, routes=routes)

In [ ]:
#export
if __name__ == '__main__':
    if 'serve' in sys.argv:
        uvicorn.run(app=app, host='0.0.0.0', port=5000, log_level="info")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 01_scraper.ipynb.
Converted 02_augmentation.ipynb.
Converted Evaluation.ipynb.
Converted ExampleBlog.ipynb.
Converted Training.ipynb.
Converted WebService.ipynb.
Converted index.ipynb.
